In [1]:
#ROI
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Load the Excel file
df = pd.read_excel('superhero_box_office.xlsx', parse_dates=['Release Date'])

# CPI data (approximate U.S. annual averages, base 2025 = 120.0)
cpi_data = {1989:46.5,1990:49.0,1991:51.1,1992:52.7,1993:54.2,1994:55.6,1995:57.2,1996:58.9,1997:60.2,1998:61.2,1999:62.5,2000:64.6,2001:66.4,
            2002:67.5,2003:69.0,2004:70.9,2005:73.3,2006:75.6,2007:77.8,2008:80.8,2009:80.5,2010:81.8,2011:84.4,2012:86.1,2013:87.4,2014:88.8,
            2015:88.9,2016:90.0,2017:92.0,2018:94.2,2019:95.9,2020:97.1,2021:101.7,2022:109.8,2023:114.4,2024:117.7,2025:120.0}

# Function to get CPI for a release year
def get_cpi(year):
    return cpi_data.get(year, cpi_data[2025])

# Adjust grosses for inflation (to 2025 dollars, in millions)
cpi_2025 = cpi_data[2025]
df['Year'] = df['Release Date'].dt.year
df['Release Date'] = df['Release Date']
df['Adjusted Profit (Millions)'] = ((df['Worldwide Revenue'] - df['Total Budget']) * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000
df['Return on Investment Percentage'] = (df['Adjusted Profit (Millions)'] / ((df['Total Budget'] * (cpi_2025 / df['Year'].apply(get_cpi))) / 1_000_000))*100
df['Movie Legs'] = (df['Legs']*10)
df['Interest Post Opening Weekend'] = (100-df['Percent of Total Gross'])


# Sort by Release Date
df = df.sort_values('Release Date')

# Create figure
fig = go.Figure()

# Unique franchises and series
franchises = ['MCU', 'DC', 'FOX', 'SONY']
unique_series = {franchise: sorted(df[df['Franchise'] == franchise]['Series'].unique()) for franchise in franchises}

# Styling of chart
colors = {
    'MCU': ('red', 'red', 'orange'),
    'DC': ('blue', 'blue', 'orange'),
    'SONY': ('black', 'black', 'orange'),
    'FOX': ('yellow', 'yellow', 'orange')
}
markers = {'MCU': 'circle', 'DC': 'square', 'SONY': 'triangle-up', 'FOX': 'diamond'}

# Add franchise traces
franchise_traces = {}
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_roi, color_leg, color_owp = colors.get(franchise, ('grey', 'darkgrey', 'lightgrey'))
    marker = markers.get(franchise, 'circle')

    franchise_traces[franchise] = []
    
    # Return on Investment
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Return on Investment Percentage'],
        mode='lines+markers',
        name=f'{franchise} ROI %',
        line=dict(color=color_roi, dash='solid'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Return on Investment Percentage'].round(0).astype(str) + '%',
        visible=True))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    
    # Opening Weekend
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Movie Legs'],
        mode='lines+markers',
        name=f'{franchise} Domestic Audience Interest',
        line=dict(color=color_leg, dash='dot'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Movie Legs'].round(0).astype(str),
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    
    # Opening Weekend Percentage
    fig.add_trace(go.Scatter(
        x=df_franchise['Release Date'],
        y=df_franchise['Interest Post Opening Weekend'],
        mode='lines+markers',
        name=f'{franchise} Interest Post Opening Weekend %',
        line=dict(color=color_owp, dash='dash'),
        marker=dict(symbol=marker),
        hoverinfo='text',
        text=df_franchise['Title'] + ': ' + df_franchise['Interest Post Opening Weekend'].round(0).astype(str) + '%',
        visible=False))
    franchise_traces[franchise].append(len(fig.data) - 1)
    

# Initialize Series traces
series_traces = {franchise: {} for franchise in franchises}
series_traces_start_index = len(fig.data)

# Add Series traces per franchise
for franchise in franchises:
    df_franchise = df[df['Franchise'] == franchise]
    color_roi, color_leg, color_owp = colors.get(franchise, ('grey', 'darkgrey', 'lightgrey'))
    marker = markers.get(franchise, 'circle')
    
    for series in unique_series[franchise]:
        df_series = df_franchise[df_franchise['Series'] == series]
        if not df_series.empty:
            series_traces[franchise][series] = []
            
            # Return on Investment
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Return on Investment Percentage'],
                mode='lines+markers',
                name=f'{series} ROI %',
                line=dict(color=color_roi, dash='solid'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Return on Investment Percentage'].round(1).astype(str) + '%',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            
            # Series Opening Weekend
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Movie Legs'],
                mode='lines+markers',
                name=f'{series} Domestic Audience Interest',
                line=dict(color=color_leg, dash='dot'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Movie Legs'].round(1).astype(str),
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            
            # Series Opening Weekend Percentage
            fig.add_trace(go.Scatter(
                x=df_series['Release Date'],
                y=df_series['Interest Post Opening Weekend'],
                mode='lines+markers',
                name=f'{series} Interest Post Opening Weekend %',
                line=dict(color=color_owp, dash='dash'),
                marker=dict(symbol=marker),
                hoverinfo='text',
                text=df_series['Title'] + ': ' + df_series['Interest Post Opening Weekend'].round(1).astype(str) + '%',
                visible=False))
            series_traces[franchise][series].append(len(fig.data) - 1)
            

# Create franchise dropdown
franchise_dropdown = [

    # ALL: Show all franchise traces
    dict(label='ALL',
        method='update',
        args=[{'visible': [True if i in [0, 3, 6, 9] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)},
        {'updatemenus[1].buttons': 
        [dict(label='None',
        method='update',
        args=[{'visible': [True if i in [0, 3, 6, 9] else False for i in range(len(franchises) * 5)] + [False] * (len(fig.data) - len(franchises) * 5)}])],
        'updatemenus[1].visible': False  # Hide series dropdown when ALL selected
        }]
        ),

    # MCU
    dict(label='MCU',
         method='update',
         args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['MCU'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['MCU'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['MCU']],
                'updatemenus[1].visible': True}]
         ),

    # DC
    dict(label='DC',
         method='update',
         args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['DC'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['DC'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['DC']],
                'updatemenus[1].visible': True}]
         ),

    # FOX
    dict(label='FOX',
         method='update',
         args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['FOX'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['FOX'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['FOX']],
                'updatemenus[1].visible': True}]
         ),

    # SONY
    dict(label='SONY',
         method='update',
         args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]},
               {'updatemenus[1].buttons': [
                   dict(label='ALL',
                        method='update',
                        args=[{'visible': [True if i in franchise_traces['SONY'] else False for i in range(len(fig.data))]}])] + [
                   dict(label=series,
                        method='update',
                        args=[{'visible': [True if i in series_traces['SONY'][series] else False for i in range(len(fig.data))]}]
                        ) for series in unique_series['SONY']],
                'updatemenus[1].visible': True}]
         )
]

# Default series dropdown (Hidden initially)
series_dropdown = [
    dict(label='None',
         method='update',
         args=[{'visible': [True] * len(franchises) * 3 + [False] * (len(fig.data) - len(franchises) * 3)}])
]

# Update layout
fig.update_layout(
    title=dict(text="Estimated Superhero Movie Return On Investment (Inflation-Adjusted 2025)", font=dict(size=40), font_family='Garamond', automargin=True, yref='container', x=0.5, xanchor='center', y=.98, yanchor='top'),
    xaxis_title='Release Date',
    yaxis=dict(title='Percentage', side='left', tickformat='.0f'),
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    updatemenus=[
        dict(
            buttons=franchise_dropdown,
            direction='down',
            showactive=True,
            x=0.15,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0
        ),
        dict(
            buttons=series_dropdown,
            direction='down',
            showactive=True,
            x=0.2,
            xanchor='left',
            y=1.05,
            yanchor='top',
            active=0,
            visible=False
        )
    ],
    xaxis=dict(range=[df['Release Date'].min() - pd.Timedelta(days=365), df['Release Date'].max() + pd.Timedelta(days=365)])
)


# Save as HTML
fig.write_html('EstimatedSuperheroMovieROI.html')